In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import Word2Vec, StringIndexer, VectorAssembler, DataFrame
from pyspark.sql.functions import col, array, explode, concat_ws, collect_list, udf, lit
import numpy as np 
from pyspark.sql.types import StringType, ArrayType, BooleanType
from datasketch import MinHash, MinHashLSH
from sklearn.cluster import KMeans
from numpy import average


# All functions used

In [36]:
def shingle(text, k):
    shingle_set = []
    for i in range(len(text)-k +1):
        shingle_set.append(text[i:i+k])
    return set(shingle_set)

def minhash_lsh(df, k_shingle,threshold):

    lsh = MinHashLSH(threshold=threshold, num_perm=128)
    minhashes = {}

    for features in df.collect():
        shingles = shingle(features["features"], k_shingle)
        m = MinHash(num_perm=128)
        for shingle_item in shingles:
            m.update(shingle_item.encode("utf8"))
        minhashes[int(features["user_id"])] = m
        lsh.insert(int(features["user_id"]), m)

    replacement_candidates = {}
    for key in lsh.keys: 
        replacement_candidates[key] = lsh.query(minhashes[key]) 

    #Key: New representative, value: Similar items
    return replacement_candidates

#Iteratively bucket unique processes together
def bucketing(replacement_candidates):
    visited_processes = set()
    new_process_dictionary = {}
    for key, values in replacement_candidates.items():
        new_values = []
        for value in values:
            if value not in visited_processes:
                visited_processes.add(value)
                new_values.append(value)
        if new_values:  # Only add non-empty lists
            new_process_dictionary[key] = sorted(new_values)
    return new_process_dictionary


def kmeans_clustering(df, n_clusters, max_iter):
    minhashes = []
    #for jaccard verification
    minhash_dict = {}
    user_ids = []
    final_buckets = {}
    for features in df.collect():
        shingles = shingle(features["features"], 5)
        m = MinHash(num_perm=128)
        for shingle_item in shingles:
            m.update(shingle_item.encode("utf8"))
        minhashes.append(m.hashvalues)
        minhash_dict[int(features["user_id"])] = m
        user_ids.append(int(features["user_id"]))

    kmeans = KMeans(n_clusters=n_clusters, max_iter=max_iter).fit(minhashes)

    user_clusters = dict(zip(user_ids, kmeans.labels_))
    final_buckets = {}
    for key, value in user_clusters.items():
        if value in final_buckets:
            final_buckets[value].append(key)
        else:
            final_buckets[value] = [key]

    return final_buckets, minhash_dict


#Get averege jaccard value per bucket

def get_averege_jaccard_sim(final_buckets, minhashes):
    sims = {}
    for key, value in final_buckets.items():
        for user_id_1 in final_buckets[key]:
            for user_id_2 in final_buckets[key]:
                if user_id_1 != user_id_2:
                    sig_1 = minhashes[int(user_id_1)]
                    sig_2 = minhashes[int(user_id_2)]
                    sim = MinHash.jaccard(sig_1, sig_2)
                    if key not in sims:
                        sims[key] = [sim]
                    else:
                        sims[key].append(sim)
    total_sum = 0
    total_count = 0
    sims = dict(sorted(sims.items()))
    for key, value in sims.items():
        avg_sim = average(value)
        print(key, avg_sim)
        total_sum += sum(value)
        total_count += len(value)
    
    overall_average = total_sum / total_count if total_count != 0 else 0
    print("Overall Average Jaccard Similarity:", overall_average)


In [3]:
spark = SparkSession.builder.appName("spark_session_1").getOrCreate()
data = spark.read.csv("data/SDG_dataset2.csv", header=True, inferSchema=True)

df = data.withColumn("arrayColumn", concat_ws("","from", "to")).withColumn("Minhash", lit(""))
df_grouped = df.groupBy("user_id").agg(
    concat_ws("",collect_list("arrayColumn")).alias("features")
)

## Parameter-tuning for k-shingles

In [27]:
replacement_candidates = minhash_lsh(df_grouped,3,0.95)
new_process_dictionary = bucketing(replacement_candidates)
print(f"Initial processes: {len(replacement_candidates)}")
print(f"After merging processes: {len(new_process_dictionary)}")

replacement_candidates = minhash_lsh(df_grouped,5,0.95)
new_process_dictionary = bucketing(replacement_candidates)
print(f"Initial processes: {len(replacement_candidates)}")
print(f"After merging processes: {len(new_process_dictionary)}")

replacement_candidates = minhash_lsh(df_grouped,7,0.95)
new_process_dictionary = bucketing(replacement_candidates)
print(f"Initial processes: {len(replacement_candidates)}")
print(f"After merging processes: {len(new_process_dictionary)}")

Initial processes: 45641
After merging processes: 8295
Initial processes: 45641
After merging processes: 12488
Initial processes: 45641
After merging processes: 33645


## Parameter tuning for Kmeans

In [34]:
from sklearn.model_selection import train_test_split, GridSearchCV

minhashes = []
user_ids = []
final_buckets = {}
for features in df_grouped.collect():
    shingles = shingle(features["features"], 5)
    m = MinHash(num_perm=128)
    for shingle_item in shingles:
        m.update(shingle_item.encode("utf8"))
    minhashes.append(m.hashvalues)
    user_ids.append(int(features["user_id"]))

param_grid = {
    'n_clusters': [100, 250, 500],
    'max_iter': [100, 500, 1000],
}

kmeans = KMeans()

grid_search = GridSearchCV(kmeans, param_grid, cv=5)

grid_search.fit(minhashes)

best_param = grid_search.best_params_
best_model = grid_search.best_estimator_

print("The best parameters are: " , best_param )
print("The best model is: ", best_model)

The best parameters are:  {'max_iter': 100, 'n_clusters': 500}
The best model is:  KMeans(max_iter=100, n_clusters=500)


## STEP 1: Find and merge

In [29]:
replacement_candidates = minhash_lsh(df_grouped,7)
new_process_dictionary = bucketing(replacement_candidates)
print(len(replacement_candidates))
print(len(new_process_dictionary))

45641
33645


## STEP 2: Find/cluster similar items

In [37]:
from sklearn.model_selection import train_test_split, GridSearchCV

users = []
for key in new_process_dictionary:
    users.append(key)

filtered_df = df_grouped[df_grouped['user_id'].isin(users)]

final_buckets, minhashes = kmeans_clustering(filtered_df,500,100)

## Verification

In [ ]:
get_averege_jaccard_sim(final_buckets, minhashes)

0 0.7235331191575753
1 0.5981104901414798
2 0.7548786739576213
3 0.7246049693108066
4 0.7294465674961934
5 0.7292580984124694
6 0.6601232361373985
7 0.7413893690403425
8 0.6009684486474105
9 0.5508145432431127
10 0.6191211480472604
11 0.6517663376386993
12 0.594710652390844
13 0.6576437698058015
14 0.6360313057382109
15 0.7691318243960862
16 0.704930922798007
17 0.6235601973968693
18 0.4993495086644125
19 0.5382667078706089
20 0.6943297092105382
21 0.6285030323178594
22 0.627957513568521
23 0.6701023343105899
24 0.5346086817110371
25 0.7443837905760302
26 0.6901287429757439
27 0.6261217212300355
28 0.567689932155658
29 0.6138171124019282
Overall Average Jaccard Similarity: 0.7063980691195367


In [50]:
# indexer = StringIndexer(inputCol="features", outputCol="from_to_type_index")
# indexed_data = indexer.fit(actual_routes_feature).transform(actual_routes_feature)
# assembler = VectorAssembler(inputCols=["from_to_type_index"], outputCol="vector")
# actual_feature_data = assembler.transform(indexed_data)

# actual_feature_data.show()
# # def is_non_zero_vector(vector):
# #     return vector.numNonzeros() > 0

# # is_non_zero_vector_udf = udf(is_non_zero_vector, BooleanType())

# # filtered_data = actual_feature_data.filter(is_non_zero_vector_udf(col("vector")))


# mh = MinHashLSH(inputCol="vector", outputCol="hashes", numHashTables=5, seed=1003)
# model = mh.fit(actual_feature_data)

# #transformed_filtered_data = model.transform(actual_feature_data).head()
# test = model.approxNearestNeighbors()

# # transformed_filtered_data.show(truncate=False, n=50)


# #similar_items.show(truncate=False)

# def is_non_zero_vector(vector):
#     return vector.numNonzeros() > 0

# from collections import defaultdict

# representative_mapping = {}

# group_mapping = defaultdict(list)

# # Iterate over the user neighbors dictionary
# for user, neighbors in new_process_dictionary.items():
#     neighbors_sorted = tuple(sorted(neighbors))
#     if neighbors_sorted in representative_mapping:
#         representative = representative_mapping[neighbors_sorted]
#     else:
#         representative = neighbors_sorted[0]
#         for neighbor in neighbors_sorted:
#             representative_mapping[neighbor] = representative
    
#     representative_mapping[user] = representative
#     group_mapping[representative].append(user)

# new_user_neighbors = {}
# for representative, users in group_mapping.items():
#     new_user_neighbors[representative] = users

# #print(new_user_neighbors)
# new_users = []
# for key, value in new_user_neighbors.items():
#     new_users.append(key)

# print(len(new_users))
# filtered_df = df_grouped[df_grouped['user_id'].isin(new_users)]
# print(f"Amount of processes: {filtered_df.count()}")
#     shingles = shingle(features["features"], 5)
#     m = MinHash(num_perm=128)
#     for shingle_item in shingles:
#         m.update(shingle_item.encode("utf8"))
#     minhashes[int(features["user_id"])] = m
#     lsh.insert(int(features["user_id"]), m)
#     neigbours = lsh.query(m)
#     print(features["user_id"], neigbours)
#     final_buckets[features["user_id"]] = neigbours

# print(final_buckets)